# Developing and testing the pryngles `cpixx` extension

In this notebook we want to test the `cpixx` extension using data collected from the application of the original `pixx` extension 

In [5]:
import pickle
import numpy as np
import pandas as pd
import os
from pryngles import pixx

## Collecting data

You must first download file `reflection-test.pkl` from Google Drive and place it at `/tmp`

In [6]:
f=open("/tmp/reflection-test.pkl","rb")
save_values=pickle.load(f)
f.close()

Now let's read the values and:

- Create a file with the fourier coefficient matrices (`<object>-ntot_nx_ny_nz.mat`)
- Create a file with the gauss integrations points (`<object>-xmu.mat`)
- Accumulate the data for a given object in a pandas dataframe.

In [7]:
matrices=dict()
columns=["delazim","beta","eta","zeta","nmugs","nmat","nfou","af","S1","S2","S3","S4"]
for i,save in enumerate(save_values):
    args=save["args"]
    obj=save["obj"].replace(" ","_")
    """
    args are:
        0: Number of points (int)
        1: Delta azim (array Nx1)
        2: betas (array: Nx1)
        3: etas (array: Nx1)
        4: zetas
        5: nmugs
        6: nmat
        7: nfou
        8: xmu
        9: rfou
        10: af
    """
    
    #Read properties
    npoints=args[0]
    delta_azim=args[1]
    betas=args[2]
    etas=args[3]
    zetas=args[4]
    nmugs,nmat,nfou=args[5:8]
    xmu=args[8]
    rfou=args[9]
    nx,ny,nz=rfou.shape
    af=args[10]
    hash_mat=hash(rfou.sum())
    
    #Read stokes
    stokes=save["stokes"]
    
    #Save in dataframe
    data=pd.DataFrame(columns=columns)
    data["delazim"]=pd.Series(delta_azim)
    data["beta"]=pd.Series(betas)
    data["eta"]=pd.Series(etas)
    data["zeta"]=pd.Series(zetas)
    data["nmugs"]=nmugs
    data["nmat"]=nmat
    data["nfou"]=nfou
    data["af"]=pd.Series(af)
    data[["S1","S2","S3","S4"]]=pd.DataFrame(stokes)
    
    #Save matrix
    if not hash_mat in matrices.keys():    
        #Save matrix
        ntot=nx*ny*nz
        filename=f"{obj}-{ntot}_{nx}_{ny}_{nz}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,rfou.reshape(ntot,1))

        filename=f"{obj}-xmu-{len(xmu)}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,xmu)

        matrices[hash_mat]=dict(
            obj=obj,
            data=pd.DataFrame(columns=columns),
            rfou=rfou,
        )
    
    #Store data
    matrices[hash_mat]["data"]=pd.concat((matrices[hash_mat]["data"],data),ignore_index=True)
    #break
        
print("Done.")

Saving file planet-5292_63_21_4.mat...
Saving file planet-xmu-21.mat...
Saving file ring_forward-414000_60_20_345.mat...
Saving file ring_forward-xmu-20.mat...
Saving file ring_back-414000_60_20_345.mat...
Saving file ring_back-xmu-20.mat...
Done.


Save the interpolation matrices (the unique values) into interpolation files:

In [8]:
for hash_mat,item in matrices.items():
    obj=item["obj"]
    print(f"Saving interpolation matrix for {obj}")
    data=item["data"]
    print(f"Original number of interpolation points {len(data)}")
    data_clean=data.drop_duplicates()
    print(f"Number of interpolation points after removing duplicates {len(data_clean)}")
    filename=f"{obj}-interpolation.mat"
    print(f"Saving file {filename}...")
    np.savetxt(filename,data_clean)

Saving interpolation matrix for planet
Original number of interpolation points 696730
Number of interpolation points after removing duplicates 193758
Saving file planet-interpolation.mat...
Saving interpolation matrix for ring_forward
Original number of interpolation points 539136
Number of interpolation points after removing duplicates 231
Saving file ring_forward-interpolation.mat...
Saving interpolation matrix for ring_back
Original number of interpolation points 616535
Number of interpolation points after removing duplicates 320
Saving file ring_back-interpolation.mat...


## Check using pixx

Read data:

In [9]:
!ls *.mat

planet-5292_63_21_4.mat		ring_back-xmu-20.mat
planet-interpolation.mat	ring_forward-414000_60_20_345.mat
planet-xmu-21.mat		ring_forward-interpolation.mat
ring_back-414000_60_20_345.mat	ring_forward-xmu-20.mat
ring_back-interpolation.mat


Read data:

In [10]:
filename="planet-5292_63_21_4.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="planet-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="planet-xmu-21.mat"
xmu=np.loadtxt(filename)
#Compare:
#rfou_ori=matrices[401387150607385127]["rfou"]
#abs(rfou-rfou_ori).mean()

Check the first column:

In [11]:
interpolation[0]

array([1.04484516e+00, 3.06939428e+00, 4.99032903e-02, 2.50967097e-02,
       2.10000000e+01, 3.00000000e+00, 3.00000000e+00, 9.43232879e-05,
       4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01])

Read the input parameter values:

In [12]:
delazim,beta,eta,zeta,nmugs,nmat,nfou,af=interpolation[0,:8]
delazim,beta,eta,zeta,nmugs,nmat,nfou,af

(1.0448451569439827,
 3.069394277348945,
 0.04990329026929557,
 0.02509670973070432,
 21.0,
 3.0,
 3.0,
 9.432328787795567e-05)

And the stokes vector components:

In [32]:
stokes=interpolation[0,8:]
stokes

array([4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01])

Let's calculate the stokes vector using the pixx.reflection routine:

In [33]:
stokes_rec=pixx.reflection(
    1,#nvals
    [delazim],
    [beta],
    [eta],
    [zeta],
    int(nmugs),
    int(nmat),
    int(nfou),
    xmu,
    rfou,
    af
)
stokes_rec

array([[4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01]])

Which coincides with what is stored:

In [34]:
stokes-stokes_rec

array([[0., 0., 0., 0.]])

Let's do it for more stored interpolation values:

In [40]:
for i in range(10):
    delazim,beta,eta,zeta,nmugs,nmat,nfou,af=interpolation[i,:8]
    stokes_sto=interpolation[i,8:]
    stokes_rec=pixx.reflection(
        1,#nvals
        [delazim],
        [beta],
        [eta],
        [zeta],
        int(nmugs),
        int(nmat),
        int(nfou),
        xmu,
        rfou,
        af
    )
    print(stokes_sto-stokes_rec)

[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]


Let's do it for all:

In [49]:
delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

In [50]:
stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )

In [51]:
stokes_sto=interpolation[:,8:]

In [53]:
abs(stokes_sto-stokes_rec).sum()

0.0

It works!

The same thing can be repeated for ring

In [54]:
filename="planet-5292_63_21_4.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="planet-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="planet-xmu-21.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

In [57]:
filename="ring_forward-414000_60_20_345.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="ring_forward-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="ring_forward-xmu-20.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

In [60]:
filename="ring_back-414000_60_20_345.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="ring_back-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="ring_back-xmu-20.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

## How reshape works

In [71]:
matrix=np.array([[[1,2,3,4],[5,6,7,8],[9,10,11,12]],[[-1,-2,-3,-4],[-6,-6,-7,-8],[-9,-10,-11,-12]]])

In [75]:
nx,ny,nz=matrix.shape
ntot=nx*ny*nz
nx,ny,nz

(2, 3, 4)

In [76]:
matrix_reshap=matrix.reshape(ntot,1)
matrix_reshap

array([[  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ -1],
       [ -2],
       [ -3],
       [ -4],
       [ -6],
       [ -6],
       [ -7],
       [ -8],
       [ -9],
       [-10],
       [-11],
       [-12]])

In [80]:
matrix_recons=np.zeros((nx,ny,nz))
n=0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            matrix_recons[i][j][k]=matrix_reshap[n]
            n+=1

In [84]:
matrix_recons

array([[[  1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.],
        [  9.,  10.,  11.,  12.]],

       [[ -1.,  -2.,  -3.,  -4.],
        [ -6.,  -6.,  -7.,  -8.],
        [ -9., -10., -11., -12.]]])